# Using Species and Mixtures

OpenPNM provides a small set of default function for computing the physical properties of pure fluids as well as mixtures.  This notebook will cover their use

In [1]:
import openpnm as op
import numpy as np

In [2]:
pn = op.network.Demo()

## Pure Species

In [3]:
A = op.phase.Species(network=pn, species='ethanol')
print(A)

ModuleNotFoundError: No module named 'thermo'

As can be seen above the ``Species`` class does not compute any properties of the given species, *BUT* it does contain a host of thermodynamic properties in the ``params`` attribute:

In [4]:
print(A.params)

NameError: name 'A' is not defined

*Pro Tip*: You can read and write values to this ``params`` attribute directly or by using ``A['params.<item>']``.  This latter option allows pore scale models to accept both ``acentric_factor='pore.acentric_factor'`` or ``acentric_factor='param.acentric_factor'``.  In both cases the function will call ``target[acentric_factor]``, but in the former case it will look in the ``target`` dictionary as normal, while in the latter case it will be redirected to the ``target.params`` dictionary. This functionality was implmented in V3 to avoid the need to store values that are truly constant in every every pore.

These parameters are all used in the various property estimation methods.  For instance, to compute the viscosity of ethanol, OpenPNM provides a function that implements the model of Steil and Thodos (``openpnm.models.phase.viscosity.liquid_pure_ls``):

In [5]:
f = op.models.phase.viscosity.liquid_pure_ls
A.add_model(propname='pore.viscosity',
            model=f)
print(A['pore.viscosity'])

NameError: name 'A' is not defined

This function requires several pieces of thermodynamics information, such as the critical temperature and pressure. You can see all the arguments below:

In [6]:
print(A.models)

NameError: name 'A' is not defined

The above shows that the temperature of the phase is fetched as ``'pore.temperature'``, while all the rest are retrieved from the ``params`` attribute.  To further illustrate this behavior, we could write the critical temperature ``A['pore.critical_temperature']`` and also overwrite the default argument:

In [7]:
A.models['pore.viscosity@all']['Tc'] = 'pore.critical_temperature'
A['pore.critical_temperature'] = A['param.critical_temperature']

NameError: name 'A' is not defined

In [8]:
print(A.models)

NameError: name 'A' is not defined

Now when we regenerate the model it will fetch the critical temperature values for each pore but will work as expected:

In [9]:
A.regenerate_models()
print(A['pore.viscosity'])

NameError: name 'A' is not defined

### Gas and Liquid Species

OpenPNM has a suite of functions for computing the properties of pure phases, but these functions differ for gases and liquids.  For this reason we offer two classes for gas and liquid with the appropriate models already defined.  These are referred to as ``StandardLiquid`` and ``StandardGas`` to indicate that the models being used are the standard selection which provide a first-approximatation:

In [10]:
A = op.phase.StandardGas(network=pn, species='o2')
B = op.phase.StandardLiquid(network=pn, species='h2o')

ModuleNotFoundError: No module named 'thermo'

These objects are populated with their respective thermodynamic properties:

In [11]:
print(A.params)

NameError: name 'A' is not defined

The models used on each can be seen by printing the models attribute:

In [12]:
print(A)

NameError: name 'A' is not defined

In [13]:
print(B)

NameError: name 'B' is not defined

## Gas and Liquid Mixtures

The framework for defining mixtures was officially added in V3, and it allows for the calculation of mixture properties based on the properties of the individual phases and their relative concentrations. There are a number of pore-scale models added to the ``openpnm.models.phase`` library for making these estimations, either using mixing rules or other correlations.

Creating a mixture first requires defining each of the pure components.  In this case, let's create vodka:

In [14]:
water = op.phase.StandardLiquid(network=pn, species='h2o')
etoh = op.phase.StandardLiquid(network=pn, species='ethanol')

ModuleNotFoundError: No module named 'thermo'

Next let's create the mixture:

In [15]:
vodka = op.phase.StandardLiquidMixture(network=pn, components=[water, etoh])

NameError: name 'water' is not defined

Before using this we must first specify the compositions:

In [16]:
vodka.x(water, 0.6)
vodka.x(etoh, 0.4)

NameError: name 'vodka' is not defined

Now we can run the models, which all require knowing the composition:

In [17]:
vodka.regenerate_models()

NameError: name 'vodka' is not defined

The main class is the ``Mixture`` class, and there are several subclasses which each add some specific functionality

The printout of a mixture includes not only the mixture properties, but also shows the individual components as well:

In [18]:
f = vodka.models['pore.heat_capacity@all']['model']

NameError: name 'vodka' is not defined

We can see that several properties for the mixture have been computed:

In [19]:
print(vodka)

NameError: name 'vodka' is not defined

If we change the composition of the components, and rerun the models, the mixture properties will change:

In [20]:
print(vodka['pore.viscosity'])
vodka.x(water, 0.8)
vodka.x(etoh, 0.2)
vodka.regenerate_models()
print(vodka['pore.viscosity'])

NameError: name 'vodka' is not defined

## Exploring the Features of the Mixture Objects

### ``info``
The info attribute reports all the existing properties on the mixture (similar to ``print``) but also of each of the components:

In [21]:
vodka.info

NameError: name 'vodka' is not defined

### Get component mole fractions

You'll notice that the mixture object has arrays called ``'pore.mole_fraction.<compname>'`` for each component. The dictionary look-up in OpenPNM will return a subdictionary if the given key is just ``'pore.mole_fraction'``.

In [22]:
vodka['pore.mole_fraction']

NameError: name 'vodka' is not defined

### ``components``
This attribute returns a dictionary with each of the components accessible by their name:

In [23]:
d = vodka.components
print(d.keys())

NameError: name 'vodka' is not defined

### ``get_comp_vals``
Since the mixture is made from several components, it is often desired get the values of a specific property from each component.  This method provides a convenient way to do this:

In [24]:
mus = vodka.get_comp_vals('pore.viscosity')
print(mus)

NameError: name 'vodka' is not defined

It is also possible to retrieve the properteis of a component by asking the mixture and appending the component name, as follows:

In [25]:
vodka['pore.viscosity.' + water.name]

NameError: name 'vodka' is not defined

In reality there is no array on ``vodka`` with the name ``'pore.viscosity.phase_01'``, but failure to find this array is what actually triggers the look-up of the array from ``water``. This is a convenient feature that is added using some 'syntactic sugar' behind the scenes in Python.  

### Using the wildcard (``*``) syntax

One more feature that has been added is the ability to fetch requested property arrays from *all* the components by replacing the component name with the universal *wildcard* symbmol: ``*`` as follows:

In [26]:
vodka['pore.viscosity.*']

NameError: name 'vodka' is not defined

Note that this returns exactly the same dictionary as the ``get_comp_vals`` method (in fact this function gets called behind the scenes), but this feature is offered for more than just convenience.  The main reason for supporting this feature is so that pore-scale models can be instructed to fetch the needed arrays for computing the mixture properties.  This is demonstrated in the following simple example of a custom mixture model:

In [27]:
def mole_fraction_weighting(phase, propname):
    xs = phase['pore.mole_fraction']
    ys = phase[propname]  # This is the key step
    z = 0.0
    for i in xs.keys():
        z += xs[i]*ys[i]
    return z

In [28]:
vals = mole_fraction_weighting(phase=vodka, propname='pore.viscosity.*')
print(vals)

NameError: name 'vodka' is not defined

The use of the ``'.*'`` as the suffix of the ``propname`` argument is crucial here.  As can be seen in the definition of ``mole_fraction_weighting``, the call to ``phase[propname]`` passes ``'pore.viscsoity.*'`` directly to the dictionary lookup of values from ``phase``, and this in turn triggers the retrieval of the ``'pore.viscosity'`` values from each component. 

If we were to pass ``'pore.viscosity'`` then the function would throw an error since the call to ``phase['pore.viscosity']`` would return a single numpy array of viscosity values of the mixture (or not find any values at all).